<a href="https://colab.research.google.com/github/Tednn493/football_match_prediction/blob/main/WebScraping_fbref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

url_standings = 'https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats'

all_matches=[]

seasons = list(range(2023,2020,-1))

for season in seasons:
  data = requests.get(url_standings)
  soup = BeautifulSoup(data.text)
  table = soup.select('table.stats_table')[0]
  links=table.find_all('a')
  links=[l.get("href") for l in links]
  links = [l for l in links if "/squads/" in l ]
  teams_url = [f'https://fbref.com{l}' for l in links]
  prev_season= soup.select("a.prev")[0].get("href")
  url_standings = f'https://fbref.com{prev_season}'

  for team_url in teams_url:
    team_name= team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    team_stats = requests.get(team_url)
    matches=pd.read_html(team_stats.text, match = "Scores & Fixtures")[0]

    soup = BeautifulSoup(team_stats.text)
    links=soup.find_all('a')
    links=[l.get("href") for l in links]
    links = [l for l in links if l and '/all_comps/shooting/' in l]
    data = requests.get(f'https://fbref.com{links[0]}')
    print(team_name)
    shooting= pd.read_html(data.text, match="Shooting")[0]
    shooting.columns=shooting.columns.droplevel()

    try:
      teamData=matches.merge(shooting[["Date",'Sh','SoT','Dist','FK','PK','PKatt']], on="Date")
    except ValueError:
      continue
    teamData=teamData[teamData["Comp"] == "Premier League"]
    teamData["Season"] = season
    teamData["Team"] = team_name
    all_matches.append(teamData)
    time.sleep(1)

match_df= pd.concat(all_matches)
match_df.to_csv("matches.csv")

In [ ]:
match_df.shape

(2280, 27)

In [ ]:

match_df.columns = [name.lower() for name in match_df.columns]
match_df.head()
match_df.to_csv("matches.csv")

In [ ]:
match_df["team"].value_counts()

Manchester City             114
Crystal Palace              114
Southampton                 114
Leeds United                114
Leicester City              114
Everton                     114
West Ham United             114
Arsenal                     114
Chelsea                     114
Wolverhampton Wanderers     114
Tottenham Hotspur           114
Aston Villa                 114
Brighton and Hove Albion    114
Liverpool                   114
Newcastle United            114
Manchester United           114
Fulham                       76
Brentford                    76
Burnley                      76
Bournemouth                  38
Nottingham Forest            38
Watford                      38
Norwich City                 38
West Bromwich Albion         38
Sheffield United             38
Name: team, dtype: int64